In [63]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [64]:
df_prices = pd.read_csv('./EDA/data/price_clean.csv')
df_sales = pd.read_csv('./EDA/data/sales_clean.csv')

In [65]:
print(df_prices.shape)
df_prices.head(1)

(1536776, 5)


,Unnamed: 0,item,store_code,date,sell_price
0,0,ACCESORIES_1_001,BOS_1,2013-07-31,11.753875


In [66]:
print(df_sales.shape)
df_sales.head(1)

(1951360, 5)


,Unnamed: 0,id,date,store,sales
0,0,ACCESORIES_1_001_BOS_1,2011-01-31,South_End,0


In [67]:
del(df_sales["Unnamed: 0"])
del(df_prices["Unnamed: 0"])

### Making new colunm so to merge by it

In [68]:
df_prices["id"] = df_prices["item"] + "_" + df_prices["store_code"]

In [69]:
df_prices.head(1)

,item,store_code,date,sell_price,id
0,ACCESORIES_1_001,BOS_1,2013-07-31,11.753875,ACCESORIES_1_001_BOS_1


### deleting useless colunms so to save RAM memory in our calculations

In [70]:
df_prices.drop(["store_code", "item"], axis = 1, inplace = True)

In [71]:
df_prices = df_prices[["id","date", "sell_price"]]

In [72]:
df_prices.head(1)

,id,date,sell_price
0,ACCESORIES_1_001_BOS_1,2013-07-31,11.753875


In [73]:
df_sales.head(1)

,id,date,store,sales
0,ACCESORIES_1_001_BOS_1,2011-01-31,South_End,0


In [74]:
df_market = pd.merge(df_sales,df_prices, on=["id", "date"], how="left")

In [75]:
print(df_market.shape)
df_market.sample(2)

(1951360, 5)


,id,date,store,sales,sell_price
452149,HOME_&_GARDEN_1_146_NYC_2,2015-06-30,Harlem,0,7.4750
618919,HOME_&_GARDEN_1_412_BOS_1,2014-04-30,South_End,11,22.4625


In [76]:
df_market["sell_price"].isnull().sum()

np.int64(414584)

### deleting nulls on sell_price column as we consider if there isn´t price it is because the item hadn´t arrive yet to the store

In [77]:
df_market.dropna(subset=(["sell_price"]), inplace = True)

In [78]:
df_market

,id,date,store,sales,sell_price
30,ACCESORIES_1_001_BOS_1,2013-07-31,South_End,10,11.753875
31,ACCESORIES_1_001_BOS_1,2013-08-31,South_End,3,10.985800
32,ACCESORIES_1_001_BOS_1,2013-09-30,South_End,4,10.985800
33,ACCESORIES_1_001_BOS_1,2013-10-31,South_End,4,10.985800
34,ACCESORIES_1_001_BOS_1,2013-11-30,South_End,10,10.985800
...,...,...,...,...,...
1951355,SUPERMARKET_3_827_PHI_3,2015-12-31,Queen_Village,41,1.200000
1951356,SUPERMARKET_3_827_PHI_3,2016-01-31,Queen_Village,49,1.200000
1951357,SUPERMARKET_3_827_PHI_3,2016-02-29,Queen_Village,46,1.200000
1951358,SUPERMARKET_3_827_PHI_3,2016-03-31,Queen_Village,68,1.200000


###  Making new columns "Income"

In [79]:
df_market["income"] = df_market["sales"] * df_market["sell_price"]

In [80]:
df_market.sample(5)

,id,date,store,sales,sell_price,income
439675,HOME_&_GARDEN_1_126_PHI_3,2015-12-31,Queen_Village,7,11.2000,78.400
1655007,SUPERMARKET_3_361_PHI_3,2013-08-31,Queen_Village,57,2.3760,135.432
119142,ACCESORIES_1_192_BOS_2,2014-03-31,Roxbury,0,4.7082,0.000
1702514,SUPERMARKET_3_436_BOS_2,2015-03-31,Roxbury,3,3.0000,9.000
1072826,SUPERMARKET_1_066_BOS_3,2015-11-30,Back_Bay,9,1.6320,14.688


### adding old columns to our df_market so to use them in visualizations or any other calculates afterwards

In [81]:
df_market["category"] = df_market["id"].apply(lambda x: x[:-12])
df_market["region"] = df_market["id"].apply(lambda x: x[-5:-2])
df_market["item"] = df_market["id"].apply(lambda x: x[0:-6])

In [82]:
df_market = df_market[['id','item','store','region','category','sell_price','sales','income']]

In [83]:
df_market.sample(2)

,id,item,store,region,category,sell_price,sales,income
1903589,SUPERMARKET_3_753_NYC_1,SUPERMARKET_3_753,Greenwich_Village,NYC,SUPERMARKET,6.576,148,973.248
1118533,SUPERMARKET_1_138_PHI_1,SUPERMARKET_1_138,Midtown_Village,PHI,SUPERMARKET,2.616,26,68.016


In [84]:
file_name = "df_market.csv" #mountly
df_market.to_csv(file_name)